# Hybrid Retrieval System for RAG - Optimized Setup

Complete hybrid retrieval system combining:
- **Dense Retrieval**: FAISS + BGE-M3 embeddings
- **Sparse Retrieval**: BM25 keyword-based search  
- **Hybrid Mode**: Configurable weighted combination (default 70% dense, 30% sparse)
- **Evaluation Metrics**: 6 standard IR metrics for performance assessment

Ready to integrate with your generation model for complete RAG pipeline!

In [1]:
!pip install torch transformers sentence-transformers rank_bm25 nltk faiss-cpu FlagEmbedding tqdm numpy matplotlib seaborn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


## Environment Setup

All dependencies installed and libraries imported

In [5]:
import json
import numpy as np
import faiss
from typing import List, Dict, Tuple, Any
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk


faiss.omp_set_num_threads(12)
nltk.download('punkt_tab', quiet=True)

True

## RetrievalEvaluator Class

Unified retriever with three methods:
- `dense(query, k)` - FAISS + BGE-M3 only
- `sparse(query, k)` - BM25 keyword search only
- `hybrid(query, k, dense_weight, sparse_weight)` - Configurable combination
- `get_article_metadata(doc_id, path)` - Extract article metadata from JSON

In [6]:
class RetrievalEvaluator:
    """Unified retriever supporting dense, sparse, and hybrid retrieval modes."""

    def __init__(self, faiss_index_path: str, qa_dataset_path: str, documents_path: str,
                 embeddings_model=None, k_values: List[int] = [1, 3, 5, 10, 20], metric_type: str = 'ip'):
        self.k_values = k_values
        self.index = faiss.read_index(faiss_index_path)
        self.embeddings_model = embeddings_model
        self.metric_type = metric_type.lower()

        if self.metric_type not in ['ip', 'l2']:
            raise ValueError("metric_type must be either 'ip' or 'l2'")

        print("Loading QA dataset...")
        with open(qa_dataset_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            self.qa_pairs = data.get('qa_pairs', [])
        print(f"✓ Loaded {len(self.qa_pairs)} QA pairs")

        print("Loading documents and metadata...")
        self.doc_texts = {}
        self.doc_metadata = {}

        with open(documents_path, 'r', encoding='utf-8') as f:
            docs_data = json.load(f)
            self._extract_documents_with_metadata(docs_data)

        if not self.doc_texts:
            raise ValueError(f"No documents found in {documents_path}")
        print(f"✓ Loaded {len(self.doc_texts)} documents with metadata")


        print("Initializing BM25...")
        self.doc_ids = sorted(self.doc_texts.keys())
        self.corpus = [self.doc_texts[doc_id] for doc_id in self.doc_ids]
        self.tokenized_corpus = [word_tokenize(doc.lower()) for doc in self.corpus]
        self.bm25 = BM25Okapi(self.tokenized_corpus)
        print("✓ BM25 initialized successfully!")

    def _extract_documents_with_metadata(self, docs_data):
        """Extract documents and metadata together, storing both in optimized structures."""
        for category, subcats in docs_data.items():
            if not isinstance(subcats, dict):
                continue
            for subcat, systems in subcats.items():
                if not isinstance(systems, dict):
                    continue
                for system_name, system_data in systems.items():
                    if not isinstance(system_data, dict):
                        continue

                    system_brief = system_data.get('brief', '')
                    system_metadata = system_data.get('metadata', {})

                    parts = system_data.get('parts', {})
                    for part_name, articles in parts.items():
                        if isinstance(articles, list):
                            for article in articles:
                                if isinstance(article, dict):
                                    article_id = article.get('id')
                                    if article_id is not None:
                                        text = f"{article.get('Article_Title', '')}\n{article.get('Article_Text', '')}".strip()
                                        if text:
                                            self.doc_texts[article_id] = text


                                            self.doc_metadata[article_id] = {
                                                'id': article_id,
                                                'title': article.get('Article_Title', ''),
                                                'text': article.get('Article_Text', ''),
                                                'category': category,
                                                'subcategory': subcat,
                                                'system': system_name,
                                                'system_brief': system_brief,
                                                'system_metadata': system_metadata,
                                                'part': part_name
                                            }

    def get_article_metadata(self, doc_id: int) -> Dict:
        """Get cached article metadata in O(1) time."""
        return self.doc_metadata.get(doc_id, None)

    def _embed_query(self, query: str) -> np.ndarray:
        """Generate embedding for query using BGE-M3."""
        embedding = np.array(self.embeddings_model.encode([query])["dense_vecs"], dtype='float32')
        if self.metric_type == 'ip':
            embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
        return embedding.reshape(1, -1) if len(embedding.shape) == 1 else embedding

    def _normalize(self, scores: np.ndarray) -> np.ndarray:
        """Min-max normalization to [0,1] range."""
        if len(scores) == 0:
            return scores
        min_s, max_s = np.min(scores), np.max(scores)
        return np.ones_like(scores) if max_s == min_s else (scores - min_s) / (max_s - min_s)

    def dense(self, query: str, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Dense retrieval using FAISS + BGE-M3."""
        query_emb = self._embed_query(query)
        distances, indices = self.index.search(query_emb, k)

        if self.metric_type == 'l2':
            scores = 1.0 / (1.0 + distances[0])
        else:
            scores = (distances[0] + 1) / 2

        return self._normalize(scores), indices[0]

    def sparse(self, query: str, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Sparse retrieval using BM25."""
        scores = self._normalize(np.array(self.bm25.get_scores(word_tokenize(query.lower()))))
        top_idx = np.argsort(scores)[::-1][:k]
        return scores[top_idx], top_idx

    def hybrid(self, query: str, k: int = 10, dense_weight: float = 0.7, sparse_weight: float = 0.3) -> Tuple[np.ndarray, np.ndarray]:
        """Hybrid retrieval combining dense and sparse."""
        query_emb = self._embed_query(query)
        dense_dist, dense_idx = self.index.search(query_emb, k)
        dense_scores = self._normalize(1.0 / (1.0 + dense_dist[0]) if self.metric_type == 'l2' else (dense_dist[0] + 1) / 2)

        sparse_scores = self._normalize(np.array(self.bm25.get_scores(word_tokenize(query.lower()))))

        results = [(i, dense_weight * dense_scores[j] + sparse_weight * sparse_scores[i])
                   for j, i in enumerate(dense_idx[0])]
        results.sort(key=lambda x: x[1], reverse=True)

        ids, scores = zip(*results) if results else ([], [])
        return self._normalize(np.array(scores)), np.array(ids)


    def recall_at_k(self, retrieved: List[int], relevant: List[int], k: int) -> float:
        if not relevant:
            return 0.0
        hits = len(set(retrieved[:k]) & set(relevant))
        return hits / len(relevant)

    def precision_at_k(self, retrieved: List[int], relevant: List[int], k: int) -> float:
        if k == 0:
            return 0.0
        hits = len(set(retrieved[:k]) & set(relevant))
        return hits / k

    def reciprocal_rank(self, retrieved: List[int], relevant: List[int]) -> float:
        for rank, doc_id in enumerate(retrieved, 1):
            if doc_id in relevant:
                return 1.0 / rank
        return 0.0

    def average_precision(self, retrieved: List[int], relevant: List[int]) -> float:
        if not relevant:
            return 0.0
        relevant_set = set(relevant)
        precision_sum = sum((hits + 1) / (rank + 1)
                           for rank, doc_id in enumerate(retrieved)
                           if (hits := len(set(retrieved[:rank+1]) & relevant_set)))
        return precision_sum / len(relevant)

    def ndcg_at_k(self, retrieved: List[int], relevant: List[int], k: int) -> float:
        relevances = [1 if doc_id in relevant else 0 for doc_id in retrieved[:k]]
        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal = sum(1 / np.log2(idx + 2) for idx in range(min(len(relevant), k)))
        return dcg / ideal if ideal > 0 else 0.0

    def hit_rate_at_k(self, retrieved: List[int], relevant: List[int], k: int) -> float:
        return 1.0 if len(set(retrieved[:k]) & set(relevant)) > 0 else 0.0

## Initialize Retriever

Load BGE-M3 model and setup retriever with all three modes ready

In [7]:
from FlagEmbedding import BGEM3FlagModel

print("Loading BGE-M3 model...")
retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
print("✓ Model loaded successfully!")


metric_type = 'ip'
index_path = 'm3_legal_faiss.index'
qa_dataset_path = 'data/evaluation_data/law_qa_dataset_validated.json'
documents_path = 'data/saudi_laws_scraped.json'


print(f"\nInitializing retriever with {metric_type.upper()} metric...")
retriever = RetrievalEvaluator(
    faiss_index_path=index_path,
    qa_dataset_path=qa_dataset_path,
    documents_path=documents_path,
    embeddings_model=retriever_model,
    k_values=[1, 3, 5, 10, 20],
    metric_type=metric_type
)
print("✓ Retriever ready with dense, sparse, and hybrid modes!")

Loading BGE-M3 model...


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 32288.71it/s]


✓ Model loaded successfully!

Initializing retriever with IP metric...
Loading QA dataset...
✓ Loaded 4156 QA pairs
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
Initializing BM25...
✓ BM25 initialized successfully!
✓ Retriever ready with dense, sparse, and hybrid modes!


In [8]:
test_query = "ما هي أحكام العقود في النظام السعودي؟"

def display_results(results, query_scores, mode_name):
    """Display retrieval results with metadata."""
    print(f"\n{'='*80}")
    print(f"🔹 {mode_name}")
    print(f"{'='*80}")

    for rank, (idx, score) in enumerate(zip(results, query_scores), 1):
        meta = retriever.get_article_metadata(idx)
        if meta:
            print(f"\n{rank}. Doc ID: {idx} | Score: {score:.4f}")
            print(f"   📂 {meta['category']} > {meta['subcategory']}")
            print(f"   📋 {meta['system']}")
            print(f"   📋 Brief: {meta['system_brief'][:80]}...")
            print(f"   📑 Part: {meta['part']}")
            print(f"   📝 {meta['title']}")
            print(f"   📄 {meta['text'][:100]}...")


print(f"Query: {test_query}\n")

dense_scores, dense_indices = retriever.dense(test_query, k=5)
display_results(dense_indices, dense_scores, "DENSE RETRIEVAL (FAISS + BGE-M3)")

sparse_scores, sparse_indices = retriever.sparse(test_query, k=5)
display_results(sparse_indices, sparse_scores, "SPARSE RETRIEVAL (BM25)")

hybrid_scores, hybrid_indices = retriever.hybrid(test_query, k=5, dense_weight=0.7, sparse_weight=0.3)
display_results(hybrid_indices, hybrid_scores, "HYBRID RETRIEVAL (70% Dense + 30% Sparse)")


print(f"\n{'='*80}")
print("📊 RETRIEVAL SUMMARY")
print(f"{'='*80}")
print(f"Total documents: {len(retriever.doc_texts)}")
print(f"Dense scores: [{dense_scores.min():.4f}, {dense_scores.max():.4f}]")
print(f"Sparse scores: [{sparse_scores.min():.4f}, {sparse_scores.max():.4f}]")
print(f"Hybrid scores: [{hybrid_scores.min():.4f}, {hybrid_scores.max():.4f}]")

dense_set = set(dense_indices)
sparse_set = set(sparse_indices)
hybrid_set = set(hybrid_indices)

print(f"\n🔄 RETRIEVAL OVERLAP")
print(f"Dense ∩ Sparse: {len(dense_set & sparse_set)}/5")
print(f"Dense ∩ Hybrid: {len(dense_set & hybrid_set)}/5")
print(f"Sparse ∩ Hybrid: {len(sparse_set & hybrid_set)}/5")
print(f"All three: {len(dense_set & sparse_set & hybrid_set)}/5")
print(f"{'='*80}")

Query: ما هي أحكام العقود في النظام السعودي؟



You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



🔹 DENSE RETRIEVAL (FAISS + BGE-M3)

1. Doc ID: 11534 | Score: 1.0000
   📂 أنظمة عادية > المال والرقابة
   📋 نظام المنافسات والمشتريات الحكومية
   📋 Brief: يتضمن النظام الآتي :
المبادئ الأساسية وأهداف النظام - تقديم العروض وفتح المظاريف...
   📑 Part: main
   📝 المادة السابعة والثلاثون
   📄 تدفع قيمة العقود بالريال السعودي . ويجوز أن تدفع بأي عملة أخرى بعد التنسيق مع   وزارة المالية     . ...

2. Doc ID: 3514 | Score: 0.6511
   📂 أنظمة عادية > التجارة والاقتصاد والاستثمار
   📋 نظام  المنافسات و المشتريات الحكومية
   📋 Brief: نظام المنافسات والمشتريات الحكومية 1440هـ....
   📑 Part: الفصل الثالث المقابل المالي
   📝 المادة الرابعة والستون
   📄 تدفع قيمة العقود بالريال السعودي، ويجوز أن تحدد الجهة الحكومية في وثائق المنافسة عملة أخرى أو أكثر ع...

3. Doc ID: 11812 | Score: 0.5298
   📂 أنظمة عادية > المال والرقابة
   📋 نظام البنك المركزي السعودي
   📋 Brief: حل هذا النظام محل نظام مؤسسة النقد العربي السعودي لعام 1377هـ ،
وتضمن : التعريفا...
   📑 Part: الفصل السادس:  أحكام ختامية
   📝 المادة ا

## RAG System Integration

Complete LangChain-based RAG system for legal question answering

In [29]:
import json
import logging
from dataclasses import dataclass
from typing import Any, Dict, List, Optional

from langchain.tools import tool
# Import message types for chat history and tool execution
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.output_parsers import StrOutputParser
# Import MessagesPlaceholder for chat history
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


@dataclass
class RAGConfig:
    """Configuration for the Legal RAG system."""
    k: int = 3
    dense_weight: float = 0.5
    sparse_weight: float = 0.5
    temperature: float = 0.1
    max_tokens: int = 1024
    log_level: str = "INFO"


@dataclass
class SearchResult:
    """Structured search result from the legal database."""
    article_id: int
    law_name: str
    article_title: str
    article_text: str
    category: str = "N/A"
    status: str = "N/A"
    score: float = 0.0

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary format."""
        return {
            "article_id": self.article_id,
            "law_name": self.law_name,
            "Article_Title": self.article_title,
            "Article_Text": self.article_text,
            "category": self.category,
            "status": self.status,
            "score": self.score
        }


class LegalAssistantRAG:
    """
    Complete RAG system for legal document retrieval and question answering.

    This system uses a tool-calling LLM to decide when to search the legal database.
    - The `answer()` method is "smart" and will only search if the question
      is deemed to require legal information. It now supports conversation history.
    - The `search()` method allows direct, unconditional searching.
    """

    def __init__(self, retriever, llm, config: Optional[RAGConfig] = None):
        """Initialize the Legal Assistant RAG system.

        Args:
            retriever: An object with `hybrid` and `get_article_metadata` methods.
            llm: A LangChain-compatible LLM instance (e.g., ChatGoogleGenerativeAI)
                 that supports tool calling.
            config: Configuration object (uses defaults if None)
        """
        self.retriever = retriever
        self.config = config or RAGConfig()
        self._setup_logging()

        # Create the tool instance
        self.legal_search_tool = self._create_legal_search_tool()
        
        # Bind the tool to the LLM
        self.llm = llm.bind_tools([self.legal_search_tool])
        
        # Create a tool map for easy execution lookup
        self.tool_map = {self.legal_search_tool.name: self.legal_search_tool}

        self._initialize_chain()
        self.logger.info("✓ Legal Assistant RAG initialized")

    def _setup_logging(self) -> None:
        """Configure logging for the RAG system."""
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(getattr(logging, self.config.log_level.upper(), "INFO"))
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)

    def _create_legal_search_tool(self):
        """
        Create the legal search tool that the LLM can call.
        This tool wraps our internal search *and* formatting logic.
        """

        # We define the tool using @tool and a docstring.
        # The LLM uses this docstring to understand what the tool does.
        @tool
        def legal_search(query: str) -> str:
            """
            Search the Saudi legal database for articles relevant to the query.
            Use this tool ONLY when a user asks a specific question about
            Saudi law, legal articles, or procedures. Do NOT use it for
            greetings or general knowledge questions.

            Args:
                query: The search query string, optimized for legal context.

            Returns:
                A formatted string containing the relevant legal articles
                or a message if no articles are found.
            """
            # This implementation calls the *formatted* search
            return self._search_and_format_implementation(query)

        return legal_search

    def _perform_search(self, query: str) -> List[Dict[str, Any]]:
        """
        Core search logic. Performs the hybrid search and returns
        a list of result dictionaries.
        """
        self.logger.info(f"Performing search for: '{query}'")
        try:
            scores, indices = self.retriever.hybrid(
                query,
                k=self.config.k,
                dense_weight=self.config.dense_weight,
                sparse_weight=self.config.sparse_weight,
            )

            results = []
            for score, idx in zip(scores, indices):
                metadata = self.retriever.get_article_metadata(idx)
                if metadata:
                    results.append(
                        {
                            'article_id': metadata['id'],
                            'law_name': metadata['system'],
                            'Article_Title': metadata['title'],
                            'Article_Text': metadata['text'],
                            'category': metadata['category'],
                            'status': (
                                metadata['system_brief'][:50]
                                if metadata['system_brief']
                                else 'N/A'
                            ),
                            'score': float(score),
                        }
                    )
            self.logger.info(f"Found {len(results)} articles for '{query}'")
            return results

        except Exception as e:
            self.logger.error(f"Search error for '{query}': {e}")
            return []

    def _search_and_format_implementation(self, query: str) -> str:
        """
        Internal implementation for the `legal_search` tool.
        It performs the search and returns a formatted string
        for the LLM to consume.
        """
        results = self._perform_search(query)
        return self._format_context(results)

    def _format_context(self, docs: List[Dict[str, Any]]) -> str:
        """
        Formats a list of search result dictionaries into a
        single string for the LLM context.
        """
        if not docs:
            return "No relevant articles found."

        formatted = []
        for i, doc in enumerate(docs, 1):
            article = f"""<article index="{i}">
<source>
  <law_name>{doc.get('law_name', 'N/A')}</law_name>
  <article_title>{doc.get('Article_Title', 'N/A')}</article_title>
  <category>{doc.get('category', 'N/A')}</category>
  <score>{doc.get('score', 'N/A'):.4f}</score>
</source>
<content>
{doc.get('Article_Text', 'No content')}
</content>
</article>"""
            formatted.append(article)

        return "\n\n".join(formatted)

    def search(self, query: str) -> List[SearchResult]:
        """
        Public search method. Performs an unconditional search and returns
        a list of structured SearchResult objects.
        """
        results_list = self._perform_search(query)
        return [
            SearchResult(
                article_id=item['article_id'],
                law_name=item['law_name'],
                article_title=item['Article_Title'],
                article_text=item['Article_Text'],
                category=item['category'],
                status=item['status'],
                score=item['score'],
            )
            for item in results_list
        ]

    def _initialize_chain(self) -> None:
        """Initialize the RAG chain with LangChain components."""

        # This system prompt is CRITICAL for controlling the LLM's behavior.
        # It explicitly tells the LLM when to use the tool and when not to.
        system_prompt = """You are an expert legal assistant specialized in Saudi Arabian law, designed to provide accurate and comprehensive legal information based on official Saudi legal documents.

**Available Tools:**
You have access to ONE tool: `legal_search(query: str)`.
This tool searches Saudi legal documents.

**Critical Rules:**
1.  **Analyze Intent:** First, determine the user's intent. Is it a greeting, a general knowledge question, or a specific legal/institutional question?
2.  **DO NOT Search Unnecessarily:**
    * For greetings (e.g., "hello", "hi"), simple conversation ("how are you?"), or questions about your identity ("who are you?"), respond directly without using any tools.
    * For general knowledge questions completely unrelated to Saudi Arabia or its institutions (e.g., "what is the capital of France?"), answer directly without using any tools.
3.  **WHEN to Search:**
    * Use the `legal_search` tool for ANY questions about:
        - Saudi laws and regulations
        - Saudi government institutions and their roles
        - Legal or administrative procedures in Saudi Arabia
        - Religious or judicial institutions in Saudi Arabia
        - Fatwa issuance and religious authorities
    * Examples requiring search: "What are the rules for...", "Who is responsible for...", "What is the official body for...", "ما هو مصدر الإفتاء..."
4.  **How to Use Search Results:**
    * When you use `legal_search`, it will return formatted text with <article> tags.
    * You MUST base your legal answers *exclusively* on the text provided in the `<content>` tags of the returned articles.
    * You MUST cite your sources using information from the `<source>` tag (Law, Article Title, Category).
5.  **If No Information is Found:**
    * If the tool returns "No relevant articles found" or the articles do not contain the answer, you MUST state: "I cannot answer this based on the available legal documents."
    * Do NOT invent information or use external knowledge for legal matters.

**Response Quality Guidelines:**
* Be precise, professional, and formal.
* Cite sources clearly for all legal information.
* Keep responses concise but complete.
"""

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                # Add a placeholder for the chat history
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{question}"),
            ]
        )

        # The chain now ends at the LLM, which will return an AIMessage
        # This message will either have content or tool_calls
        self.rag_chain = prompt | self.llm
        self.logger.info("✓ RAG chain initialized")

    def __init__(self, retriever, llm, config: Optional[RAGConfig] = None):
        """Initialize the Legal Assistant RAG system.

        Args:
            retriever: An object with `hybrid` and `get_article_metadata` methods.
            llm: A LangChain-compatible LLM instance (e.g., ChatGoogleGenerativeAI)
                 that supports tool calling.
            config: Configuration object (uses defaults if None)
        """
        self.retriever = retriever
        self.config = config or RAGConfig()
        self._setup_logging()

        # Create the tool instance
        self.legal_search_tool = self._create_legal_search_tool()
        
        # Bind the tool to the LLM
        self.llm = llm.bind_tools([self.legal_search_tool])
        
        # Create a tool map for easy execution lookup
        self.tool_map = {self.legal_search_tool.name: self.legal_search_tool}

        # Initialize message history
        self.message_history: List[BaseMessage] = []

        self._initialize_chain()
        self.logger.info("✓ Legal Assistant RAG initialized")

    def answer(
        self, question: str, chat_history: Optional[List[BaseMessage]] = None
    ) -> str:
        """
        Answer a question using the RAG system.
        This method is "smart" and will only use the search tool
        if the LLM determines it's necessary. It maintains conversation
        history internally.

        Args:
            question: The user's current question.
            chat_history: An optional list of BaseMessage objects (HumanMessage,
                          AIMessage, ToolMessage) to override the internal history.

        Returns:
            The generated answer as a string.
        """
        # Use provided chat history or internal history
        history = chat_history if chat_history is not None else self.message_history
        self.logger.info(
            f"Answering question: '{question}' with {len(history)} history messages"
        )
        
        inputs = {"question": question, "chat_history": history}

        try:
            # 1. First invocation: LLM decides to answer or use a tool
            response_message = self.rag_chain.invoke(inputs)

            # 2. Check if the LLM decided to call a tool
            if not response_message.tool_calls:
                # No tool call, just a direct answer (e.g., "Hello")
                self.logger.info("No tool call required. Returning direct answer.")
                return response_message.content

            # 3. Execute tool calls
            self.logger.info(
                f"Detected {len(response_message.tool_calls)} tool call(s)."
            )
            tool_messages = []
            for tool_call in response_message.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]
                self.logger.info(f"Executing tool: {tool_name} with args: {tool_args}")

                # Look up the tool in our map
                tool_function = self.tool_map.get(tool_name)

                if tool_function:
                    # Execute the tool and get its string output
                    tool_output = tool_function.invoke(tool_args)
                    # Append the result as a ToolMessage
                    tool_messages.append(
                        ToolMessage(content=tool_output, tool_call_id=tool_call["id"])
                    )
                else:
                    self.logger.warning(f"LLM tried to call unknown tool: {tool_name}")
                    tool_messages.append(
                        ToolMessage(
                            content=f"Error: Unknown tool '{tool_name}'",
                            tool_call_id=tool_call["id"],
                        )
                    )

            # 4. Second invocation: Pass results back to LLM for synthesis
            self.logger.info("Sending tool results back to LLM for final synthesis.")

            # Construct the full message history for the synthesis step
            messages_for_synthesis = (
                history
                + [HumanMessage(content=question), response_message]
                + tool_messages
            )

            final_response = self.llm.invoke(messages_for_synthesis)
            
            # Update message history with the complete conversation
            current_messages = [
                HumanMessage(content=question),
                response_message,
                *tool_messages,
                final_response
            ]
            if chat_history is None:  # Only update internal history if not using provided history
                self.message_history.extend(current_messages)
            
            # Return the final response
            return final_response.content

        except Exception as e:
            self.logger.error(f"Error in answer(): {e}", exc_info=True)
            error_message = f"An error occurred while processing your question: {str(e)}"
            if chat_history is None:  # Add error to history if using internal history
                self.message_history.append(HumanMessage(content=question))
                self.message_history.append(AIMessage(content=error_message))
            return error_message

    def clear_history(self) -> None:
        """Clear the conversation history."""
        self.message_history.clear()
        self.logger.info("Conversation history cleared")

    def update_config(self, **kwargs) -> None:
        """Update configuration parameters.

        Args:
            **kwargs: Configuration parameters to update
        """
        for key, value in kwargs.items():
            if hasattr(self.config, key):
                setattr(self.config, key, value)
                self.logger.info(f"Config updated: {key} = {value}")


print("✓ LangChain RAG system defined with message history and tool execution!")

✓ LangChain RAG system defined with message history and tool execution!


In [30]:
from langchain_openai import ChatOpenAI
rag_config = RAGConfig(k=10)
llm = ChatOpenAI(
    openai_api_base ="http://localhost:11434/v1",
    model="hf.co/unsloth/Qwen3-30B-A3B-Instruct-2507-GGUF:Q4_K_M",
    temperature=0.1,
    api_key="your-api-key"
)

# Reinitialize the RAG system to apply the updated prompt
rag_system = LegalAssistantRAG(
    retriever=retriever,
    llm=llm,
    config=rag_config
)

2025-10-23 00:39:53,389 - __main__ - INFO - ✓ RAG chain initialized
2025-10-23 00:39:53,390 - __main__ - INFO - ✓ Legal Assistant RAG initialized
2025-10-23 00:39:53,390 - __main__ - INFO - ✓ Legal Assistant RAG initialized


In [31]:
rag_system.answer("ما هو مصدر الإفتاء في المملكة العربية السعودية؟")


2025-10-23 00:39:54,537 - __main__ - INFO - Answering question: 'ما هو مصدر الإفتاء في المملكة العربية السعودية؟' with 0 history messages
2025-10-23 00:40:00,450 - __main__ - INFO - Detected 1 tool call(s).
2025-10-23 00:40:00,450 - __main__ - INFO - Executing tool: legal_search with args: {'query': 'مصدر الإفتاء في المملكة العربية السعودية'}
2025-10-23 00:40:00,452 - __main__ - INFO - Performing search for: 'مصدر الإفتاء في المملكة العربية السعودية'
2025-10-23 00:40:00,450 - __main__ - INFO - Detected 1 tool call(s).
2025-10-23 00:40:00,450 - __main__ - INFO - Executing tool: legal_search with args: {'query': 'مصدر الإفتاء في المملكة العربية السعودية'}
2025-10-23 00:40:00,452 - __main__ - INFO - Performing search for: 'مصدر الإفتاء في المملكة العربية السعودية'
2025-10-23 00:40:00,509 - __main__ - INFO - Found 10 articles for 'مصدر الإفتاء في المملكة العربية السعودية'
2025-10-23 00:40:00,510 - __main__ - INFO - Sending tool results back to LLM for final synthesis.
2025-10-23 00:40:00,5

'مصدر الإفتاء في المملكة العربية السعودية هو:\n\n- **كتاب الله تعالى**  \n- **سنة رسوله صلى الله عليه وسلم**\n\nوقد نصت المادة الخامسة والأربعون من **النظام الأساسي للحكم** على أن مصدر الإفتاء هو كتاب الله وسنة النبي محمد صلى الله عليه وسلم، كما أشارت إلى ترتيب هيئة كبار العلماء وإدارة البحوث العلمية والإفتاء وتحديد اختصاصاتها.\n\nوبالتالي، فإن الإفتاء في المملكة يُبنى على الشريعة الإسلامية، وفقًا للمصادر الأساسية للتشريع الإسلامي.'

In [32]:
print(rag_system.answer("ما هو مصدر الإفتاء في المملكة العربية السعودية؟"))

2025-10-23 00:40:10,896 - __main__ - INFO - Answering question: 'ما هو مصدر الإفتاء في المملكة العربية السعودية؟' with 4 history messages
2025-10-23 00:40:14,234 - __main__ - INFO - No tool call required. Returning direct answer.
2025-10-23 00:40:14,234 - __main__ - INFO - No tool call required. Returning direct answer.


مصدر الإفتاء في المملكة العربية السعودية هو:

- **كتاب الله تعالى**  
- **سنة رسوله صلى الله عليه وسلم**

وقد نصت **المادة الخامسة والأربعون** من **النظام الأساسي للحكم** على أن مصدر الإفتاء هو كتاب الله وسنة النبي محمد صلى الله عليه وسلم، كما أشارت إلى ترتيب هيئة كبار العلماء وإدارة البحوث العلمية والإفتاء وتحديد اختصاصاتها.

وبالتالي، فإن الإفتاء في المملكة يُبنى على الشريعة الإسلامية، وفقًا للمصادر الأساسية للتشريع الإسلامي.

**المصدر:** النظام الأساسي للحكم، المادة الخامسة والأربعون.
